In [1]:
import pandas as pd

In [2]:
import opensmile
import os
import requests
import time
# https://github.com/audeering/opensmile-python
smile = opensmile.Smile(
    # feature_set=opensmile.FeatureSet.ComParE_2016,
    # feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_set=opensmile.FeatureSet.emobase,
    feature_level=opensmile.FeatureLevel.Functionals,
)


def error(id, e):
    content = '[{}]\t{}'.format(str(id), str(e))
    with open(r'./audio_process_error.log', 'a') as f:
        f.write('{}\n'.format(content))


def sleep():
    print('sleeping...')
    print()
    time.sleep(15*60)


def process(url, id):
    filename = './tmp/{}.mp3'.format(id)
    state = 1
    trytime = 3
    while trytime > 0 and state != 0:
        try:
            # audio = requests.get(url)
            with requests.Session() as s:
                audio = s.get(url)
            with open(filename, 'wb') as f:
                f.write(audio.content)
            state = 0
        except Exception as e:
            if 'Temporary failure in name resolution' in str(e):
                # print(id, e)
                error(id, e)
                sleep()
                break
            if 'timeout' not in str(e):
                # print(id, e)
                error(id, e)
                sleep()
                break
            # print(id, e)
            error(id, e)
            sleep()
            state = 1
            trytime -= 1
    try:
        if os.path.exists(filename):
            processed = smile.process_file(filename)
            os.remove(filename)
            processed['track_id'] = id
            # result = {'col':processed.columns.tolist(), 'data': processed.iloc[0].tolist()}
            processed.index = [0]
            return processed
        else:
            return None
    except Exception as e:
        # print(id, e)
        error(id, e)
        # raise e
    return None


SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


OSError: /lib64/libm.so.6: version `GLIBC_2.27' not found (required by /home/alan/anaconda3/lib/python3.7/site-packages/opensmile/core/bin/linux/libSMILEapi.so)